In [2]:
using CSV
using DataFrames
using FileIO

In [36]:
a = [1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [39]:
[4; a]

4-element Vector{Int64}:
 4
 1
 2
 3

In [62]:
function write_merged_spec(save_name, specs)
#     @show save_name
#     @show specs
    open(save_name, "w") do io
        write(io, "(declare-const X_0 Real)\n")
        write(io, "(declare-const Y_0 Real)\n")
        write(io, "(assert (or\n")
        for i in 1:length(specs)
            write(io, "    (and (>= X_0 "*string(specs[i][1])*") (<= X_0 "*string(specs[i][2])*") (> Y_0 "*string(specs[i][3])*"))\n")
        end
        write(io, "))\n")
    end
end

function read_spec(spec_name)
    open(spec_name, "r") do io
        readline(io)
        readline(io)
        x1 = parse(Float64, split(readline(io)[1:end-2], " ")[end])
        x2 = parse(Float64, split(readline(io)[1:end-2], " ")[end])
        y1 = parse(Float64, split(readline(io)[1:end-2], " ")[end])
        y2 = parse(Float64, split(readline(io)[1:end-2], " ")[end])
        return [x1, x2, y2]
    end
end
function merge(csv_name, sep=5)
    df = CSV.read(csv_name,DataFrame, header=false)
    specs = []
    save_name = []
    spec_nets = []
    spec_names = []
    spec_times = []
    times = []
    for i in 1:size(df)[1]
        sp = split(df[i,2][1:end-7],"_")
        tot_idx = parse(Int64, sp[end-1])
        spec_idx = parse(Int64, sp[end])
        if spec_idx > tot_idx
            break #violation case
        end
#         @show df[i,3]
#         @show read_spec(df[i,2])
        push!(specs, read_spec(df[i,2]))
        push!(save_name, string(spec_idx))
        push!(times, df[i,3])
        if mod(i,sep) == 0
            save_name = join([sp[1]; sp[2]; save_name;], '_')*".vnnlib"
            write_merged_spec(save_name, specs)
            push!(spec_names, save_name)
            push!(spec_times, sum(times))
            push!(spec_nets, df[i,1])
            save_name = []
            specs = []
            times = []
        end
    end
    csv_name = csv_name[1:end-4] * "_disjunction" * csv_name[end-3:end]
    @show csv_name
    df = DataFrame(net = spec_nets, spec = spec_names, time = spec_times)
    CSV.write(csv_name, df, header=false) 
end

merge (generic function with 2 methods)

In [63]:
merge("normal_100.csv", 5)
merge("normal_1000.csv", 50)
merge("lognormal_100.csv", 5)
merge("lognormal_1000.csv", 50)
merge("piecewise_100.csv", 5)
merge("piecewise_1000.csv", 50)

csv_name = "normal_100_disjunction.csv"
csv_name = "normal_1000_disjunction.csv"
csv_name = "lognormal_100_disjunction.csv"
csv_name = "lognormal_1000_disjunction.csv"
csv_name = "piecewise_100_disjunction.csv"
csv_name = "piecewise_1000_disjunction.csv"


"piecewise_1000_disjunction.csv"